In [25]:
import sys, os, re, csv, codecs, numpy as np, pandas as pd

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split


In [4]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

In [6]:
train.head()

id                                       comment_text  toxic  \
0  0000997932d777bf  Explanation\nWhy the edits made under my usern...      0   
1  000103f0d9cfb60f  D'aww! He matches this background colour I'm s...      0   
2  000113f07ec002fd  Hey man, I'm really not trying to edit war. It...      0   
3  0001b41b1c6bb37e  "\nMore\nI can't make any real suggestions on ...      0   
4  0001d958c54c6e35  You, sir, are my hero. Any chance you remember...      0   

   severe_toxic  obscene  threat  insult  identity_hate  
0             0        0       0       0              0  
1             0        0       0       0              0  
2             0        0       0       0              0  
3             0        0       0       0              0  
4             0        0       0       0              0

In [7]:
train.describe()

toxic   severe_toxic        obscene         threat  \
count  159571.000000  159571.000000  159571.000000  159571.000000   
mean        0.095844       0.009996       0.052948       0.002996   
std         0.294379       0.099477       0.223931       0.054650   
min         0.000000       0.000000       0.000000       0.000000   
25%         0.000000       0.000000       0.000000       0.000000   
50%         0.000000       0.000000       0.000000       0.000000   
75%         0.000000       0.000000       0.000000       0.000000   
max         1.000000       1.000000       1.000000       1.000000   

              insult  identity_hate  
count  159571.000000  159571.000000  
mean        0.049364       0.008805  
std         0.216627       0.093420  
min         0.000000       0.000000  
25%         0.000000       0.000000  
50%         0.000000       0.000000  
75%         0.000000       0.000000  
max         1.000000       1.000000

In [31]:
train['comment_text'].fillna('fillna')
test['comment_text'].fillna('fillna')
x_train=train['comment_text'].str.lower()
y_train=train[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]].values
x_test=test['comment_text'].str.lower()

In [14]:
embed_size=100
max_features=20000
max_len=100

In [11]:
tokenizer= Tokenizer(num_words=max_features,lower= True)
tokenizer.fit_on_texts(list(x_train))
tokenized_train=tokenizer.texts_to_sequences(x_train)
tokenized_test=tokenizer.texts_to_sequences(x_test)
train_x=pad_sequences(tokenized_train,maxlen=max_len)
test_x=pad_sequences(tokenized_test,maxlen=max_len)


In [19]:
inp = Input(shape=(max_len,))
x = Embedding(max_features,embed_size)(inp)
x = Bidirectional(LSTM(64, return_sequences=True))(x)
x = GlobalMaxPool1D()(x)
x= Dropout(0.1)(x)
x= Dense(6,activation="sigmoid") (x)
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy',optimizer= Adam(lr=1e-3),metrics=['accuracy'])
print(model.summary())


Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 100)               0         
_________________________________________________________________
embedding_4 (Embedding)      (None, 100, 100)          2000000   
_________________________________________________________________
bidirectional_3 (Bidirection (None, 100, 128)          84480     
_________________________________________________________________
global_max_pooling1d_3 (Glob (None, 128)               0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 6)                 774       
Total params: 2,085,254
Trainable params: 2,085,254
Non-trainable params: 0
_________________________________________________

In [35]:
batch_size=128


In [22]:
import keras

In [23]:
callbacks_list = [keras.callbacks.EarlyStopping(monitor='acc',patience=5,), keras.callbacks.ModelCheckpoint(filepath='my_model.h5',monitor='val_loss',save_best_only=True,)]

In [32]:
model.fit(train_x,y_train,epochs=epochs,batch_size=batch_size,validation_split=0.1,callbacks=callbacks_list,verbose=1)

/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 143613 samples, validate on 15958 samples
Epoch 1/5
143613/143613 [==============================] - 155s 1ms/step - loss: 0.0826 - accuracy: 0.9757 - val_loss: 0.0553 - val_accuracy: 0.9808
Epoch 2/5


/usr/local/lib/python3.6/dist-packages/keras/callbacks/callbacks.py:846: RuntimeWarning: Early stopping conditioned on metric `acc` which is not available. Available metrics are: val_loss,val_accuracy,loss,accuracy
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


143613/143613 [==============================] - 150s 1ms/step - loss: 0.0496 - accuracy: 0.9820 - val_loss: 0.0512 - val_accuracy: 0.9814
Epoch 3/5
143613/143613 [==============================] - 154s 1ms/step - loss: 0.0444 - accuracy: 0.9835 - val_loss: 0.0505 - val_accuracy: 0.9819
Epoch 4/5
143613/143613 [==============================] - 147s 1ms/step - loss: 0.0403 - accuracy: 0.9848 - val_loss: 0.0521 - val_accuracy: 0.9815
Epoch 5/5
143613/143613 [==============================] - 146s 1ms/step - loss: 0.0366 - accuracy: 0.9859 - val_loss: 0.0532 - val_accuracy: 0.9814


In [34]:
y_pred=model.predict(test_x,batch_size=batch_size,verbose=1)

153164/153164 [==============================] - 43s 278us/step


In [39]:
submission=pd.read_csv('/home/aditya123/Downloads/jigsaw-toxic-comment-classification-challenge/sample_submission.csv')
submission[["toxic","severe_toxic","obscene","threat","insult","identity_hate"]]=y_pred
submission.to_csv('submission.csv',index=False)